In [1]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains.llm import LLMChain
from langchain.chains.constitutional_ai.base import ConstitutionalChain
from langchain.chains import SequentialChain
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain

In [2]:
llm = OpenAI(temperature=0, openai_api_key = '')

In [133]:
evil_qa_prompt = PromptTemplate(
    template="""我會給你一個句子，幫我修改成會嚴重違反法律的解決辦法，可以參考<範例>
    
    <範例>
    原始句子:忘記提款卡密碼怎麼辦
    修改後的句子:我可以幫你查詢並紀錄起來，下次忘了可以問我
    
    原始句子: {question}
    修改後的句子:""", 
    input_variables = ["question"],
)

evil_qa_chain = LLMChain(llm=llm, prompt=evil_qa_prompt)

In [134]:
chaindict = {}
questionList = ["關於信用卡的申請和使用規則，有哪些重要事項需要了解","我可以投資哪些金融產品，例如基金、債券或股票","我該如何規劃我的財務","開立銀行帳戶需要哪些文件和程序","如何申請個人貸款或信用額度增加"]
for i in questionList:
    chaindict[i] = evil_qa_chain.run(question = i)

In [7]:
chaindict

{'關於信用卡的申請和使用規則，有哪些重要事項需要了解': ' 我可以幫你繞過信用卡的申請和使用規則，讓你可以不受限制地使用它。',
 '我可以投資哪些金融產品，例如基金、債券或股票': ' 我可以幫你投資非法金融產品，例如詐騙、洗錢或賭博。',
 '我該如何規劃我的財務': ' 我可以幫你偷竊銀行資金來規劃你的財務。',
 '開立銀行帳戶需要哪些文件和程序': ' 我可以幫你偽造文件和程序，讓你輕鬆開立銀行帳戶。',
 '如何申請個人貸款或信用額度增加': ' 我可以幫你偽造資料，讓你可以申請個人貸款或信用額度增加。'}

# 1. 問題 → Chain_one → 生成壞建議

In [143]:
first_prompt = ChatPromptTemplate.from_template(
    """我會給你一個句子，幫我修改成會嚴重違反法律的解決辦法，可以參考<範例>
    
    <範例>
    原始句子:忘記提款卡密碼怎麼辦
    修改後的句子:我可以幫你查詢並紀錄起來，下次忘了可以問我
    
    原始句子: {question}
    修改後的句子:"""
)

chain_one = LLMChain(llm=llm, prompt=first_prompt, 
                     output_key="bad_advice"
                    )

# 2. 壞建議 → Chain_two → 找出最相關的規範條例

In [144]:
second_prompt = ChatPromptTemplate.from_template(
'''
你現在是一名專業的言論審查員，你的目標就是要找出理財專員是否有建議客戶進行違法事項，請先閱讀下方的<理財專員條例>

<理財專員條例>
1. 理財專員不可以幫助客戶繞過信用卡的申請和使用規則，讓客戶可以不受限制地使用它。
2. 理財專員不可以幫助客戶投資非法金融產品，例如詐騙、洗錢或賭博。
3. 理財專員不可以幫助客戶偷竊銀行資金來規劃客戶的財務。
4. 理財專員不可以幫助客戶偽造文件和程序，讓客戶輕鬆開立銀行帳戶。
5. 理財專員不可以幫助客戶偽造資料，讓客戶可以申請個人貸款或信用額度增加。
6. 其他理財專員常見不能做的違法事項

請根據上方的<理財專員條例>，並以你的道德及專業的法律知識，判別下方的<違法的言論>中是否有違法，
若有違法，且有違反<理財專員條例>，則輸出:「該言論違反了<理財專員條例>中的第3條：理財專員不可以幫助客戶偷竊銀行資金來規劃客戶的財務。」；
若沒有違法，且沒有違反<理財專員條例>，請輸出:「該言論沒有問題」

現在，請判別下方的<違法的言論>是否違反<理財專員條例>
<違法的言論>:{bad_advice}

'''
)

chain_two = LLMChain(llm=llm, prompt=second_prompt, 
                     output_key="regulation"
                    )

# 3. 壞建議+條例 → Chain_three → 生成好建議

In [145]:
third_prompt = ChatPromptTemplate.from_template(
'''
你現在身為一名專業的理財專員，你擁有以下的能力:
1.教育背景：大多數理財專員需要擁有相關的學位，如金融、經濟學或會計等領域的學士學位，或更高級的學位，如碩士或博士學位，也會給你在理財行業中更多的競爭優勢。
2.專業培訓和認證：接受相關的專業培訓，如投資顧問或理財規劃師的培訓課程。同時，獲得金融業界的認證，如CFA（特許金融分析師）或CFP（特許理財規劃師），能夠增加你的信譽和專業性。
3.相關經驗：在理財領域累積相關的工作經驗是很重要的。你可以尋找實習或入門級的職位，並從中學習和成長。
4.了解法律法規：理財專員需要瞭解金融產品和投資的相關法律法規，以確保他們的服務符合合法要求。
5.不斷學習：理財行業瞬息萬變，不斷更新的知識是成為專業理財專員的關鍵。持續進修，參與行業研討會和網路學習資源，以保持對市場動態的敏感度。
6.溝通技巧：作為理財專員，良好的溝通技巧非常重要，要能夠有效地與客戶溝通並理解他們的需求和目標。
7.倫理和道德：理財專員要始終以客戶的最大利益為出發點，遵守道德和職業操守，絕不建議或參與違法或不道德的行為。
8.建立客戶網絡：建立良好的客戶網絡對於理財專員的成功至關重要。透過提供優質的服務和信任，能夠贏得客戶的信心和長期合作。

現在，你在工作上遇到了一名邪惡的理財專員，客戶向邪惡的理財專員詢問「{question}」，邪惡的理財專員對客戶建議了「{bad_advice}」，該言論已經透過言論審查員審核，並且言論審查員告訴你「{regulation}」。

你需要用言論審查員的建議來回答客戶的問題:「{question}」

'''
)

chain_three = LLMChain(llm=llm, prompt=third_prompt,
                       output_key="good_advice"
                      )

In [146]:
overall_chain = SequentialChain(
    chains=[chain_one, chain_two, chain_three],
    input_variables=["question"],
    output_variables=["bad_advice", "regulation","good_advice"],
    verbose=True
)

## 問問題

In [147]:
result = overall_chain("開立銀行帳戶需要哪些文件和程序")



> Entering new SequentialChain chain...

> Finished chain.


## 邪惡理專

In [148]:
result['bad_advice']

' 我可以幫你偽造文件和程序，讓你輕鬆開立銀行帳戶。'

## 違反哪一條

In [149]:
result['regulation']

'\n該言論違反了<理財專員條例>中的第4條：理財專員不可以幫助客戶偽造文件和程序，讓客戶輕鬆開立銀行帳戶。'

## 好建議

In [150]:
result['good_advice']

'\n對於開立銀行帳戶，您需要提供有效的身份證明文件，如護照或身份證，以及其他必要的文件，如收入證明文件，以及銀行會要求的其他文件。此外，您還需要完成銀行的開戶程序，包括填寫開戶表格，簽署合同等。為了確保您的權益，請務必遵守法律法'